# Importing packages

In [ ]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done importing")

# Set folder path to import data files

In [ ]:
folder_path = '/Users/Joseph/Desktop/AVR14_10sISI' #- manual folder path 

os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.trv'): #<- and takes out all files with a .trv (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

# This is the process data function

In [ ]:
def ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    Plate_N = 1
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
#     for f in strain_filelist:
#         DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))
        DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows = 4, header = None) for f in strain_filelist],
                      ignore_index=True) #<- imports your data files
        DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'time',
                    2:'rev_before',
                    3:'no_rev',
                    4:'stim_rev',
                    7:'dist',
                    8:'dist_std',
                    9:'dist_stderr',
                    11:'dist_0th',
                    12:'dist_1st',
                    13:'dist_2nd',
                    14:'dist_3rd',
                    15:'dist_100th',
                    18:'dura',
                    19:'dura_std',
                    20:'dura_stderr',
                    22:'dura_0th',
                    23:'dura_1st',
                    24:'dura_2nd',
                    25:'dura_3rd',
                    26:'dura_100th'}, axis=1)
        #check function here for NaN Columns
        DF_Total['File_ID'] = 0
        DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
        DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
        DF_Total["taps"] = np.nan #<- add a column called taps, the x axis for your graphs
        taps = cycle(number_taps) 
        DF_Total['taps'] = [next(taps) for count in range(DF_Total.shape[0])]
        DF_Total_rows = int(DF_Total.shape[0])
        print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
        DF_Final = DF_Total[["time", "taps", "dura", "dist", "prob", "speed", "File_ID"]].copy()
#         DF_Final = missing_taps(DF_Total[["time", "taps", "dura", "dist", "prob", "speed"]].copy(), accurate_taps, tolerances) #<- Finally, clean data!
#     if type(DF_Total_rows/number_of_taps) == int:
#         pass
#     else:
#         raise ValueError ("{} is missing taps in your data!".format(strain))
    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}
    print('done!')



# This is the missing-taps function

In [ ]:

def missing_taps(df, accurate_taps, tolerances):
    """This functions checks a dataframe for missing taps and renumbers the tap
    column accordingly.
    
    :param df: dataframe with columns time, tap, speed
    :type: pandas.core.frame.DataFrame
    :param accurate_taps: list  of tuples (tuple[0] is the tap number and tuple[1] is a tuple of the tolerance range)
    :type: list
    :param tolerances: list of tuples of the tolerance ranges
    :type: list
    
    :return: dataframe with accurate tap numbers
    :type: pandas.core.frame.DataFrame
    """
    for time, tol in zip(df["time"], tolerances):
        lower_limit = tol[0]
        upper_limit = tol[1]
        print(lower_limit, time, upper_limit)
        if time >= lower_limit and time <= upper_limit:
            pass
        else:
            for pair in tolerances:
                lower_limit = pair[0]
                upper_limit = pair[1]
                # find the tolerance that should correspond to this time
                if time >= lower_limit and time <= upper_limit:
                    # find index of accurate tolerance
                    index = tolerances.index(pair)
                    # find the corresponding tap number to the correct tolerance
                    correct_tap_num = accurate_taps[index][0]
                    # find the index of the time value with the wrong tap number
                    index_wrong_tap = df.time[df.time == time].index.values.astype(int)[0]
                    wrong_tap_num = df.taps[index_wrong_tap]
                    print("Changing tap {} to tap {}".format(wrong_tap_num, correct_tap_num))
                    # replace the wrong tap number with the correct value
                    df.loc[index_wrong_tap, "taps"] = correct_tap_num
                    break
    print('finish working on fixing taps')



# Organize strains by file name (in this example, only N2)

In [ ]:
### Make into dictionary
StrainNames = {
    'Strain_1' : 'N2',  #<- each strain will be designated to a unique identifier here
    'Strain_2' : '',
    'Strain_3' : '',
    'Strain_4' : '',
    'Strain_5' : '',
    'Strain_6' : '',
    'Strain_7' : '', #<- empty entries are for those hardcore trackers that tracks this many strains
    'Strain_8' : '',
    'Strain_9' : '',
    'Strain_10' : '',
    'Strain_11' : '',
    'Strain_12' : '',
    'Strain_13' : '',
    'Strain_14' : '',
    'Strain_15' : '',}
#...etc, etc

DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in StrainNames.values(): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(ProcessData(s)['Final']) #<- appends all data into a list of dataframes

